In [ ]:
#Aula 4 - Exercídio de Classificação com kNN

In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import DistanceMetric
from sklearn import preprocessing

In [80]:
p = '### Predicting... ###'

In [81]:
class KNNClassifier(object):
    def __init__(self):
        self.X_train = None
        self.Y_train = None
        self.k = 1
        self.distance = 'euclidean'

    def dist_q(self, a, b):
        d = DistanceMetric.get_metric(self.distance)
        matD = d.pairwise([a,b])
        return matD[0,-1]

    def closest(self, row):
        dd = [self.dist_q(row, item) for _,item in self.X_train.iterrows()]
        v = sorted(dd)[:self.k]
       
        
        n = [dd.index(x) for x in v]
       
        vt = self.Y_train.iloc[n]
        vt = np.array(vt)
        
        label = np.argmax(np.bincount(vt))
        return label

    def fit(self, training_data, training_labels, k=1, distance='euclidean'):
        self.X_train = training_data
        self.Y_train = training_labels
        self.k = k
        self.distance = distance

    def predict(self, to_classify):
        print(p)
        pr = []
        for _,row in to_classify.iterrows():
            label = self.closest(row)
           
            pr.append(label)
        return pr

In [85]:
dataset = pd.read_csv('train.csv')

#Remoção de elementos não relevantes
del dataset['Cabin'] 
del dataset['Ticket'] 
del dataset['PassengerId'] 
del dataset['Name']

#Mapemaneto / valores NaN
dataset['Age'] = dataset.Age.fillna(dataset.Age.mean())
dataset = dataset.where((pd.notnull(dataset)), 0)
for row in ["Sex", "Embarked"]:
    dataset[row] = dataset[row].astype('category')
    dataset[row] = dataset[row].cat.codes
datasetCopy = dataset.copy()

#Separando o dataset para teinamento
list(dataset)
Y = dataset['Survived'].copy()
del dataset['Survived']
X = dataset

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.3)



In [86]:
knn = KNNClassifier()
knn.fit(X_train, Y_train, k=10, distance='minkowski')
res = knn.predict(X_test)
sc = metrics.accuracy_score(y_pred = res, y_true = Y_test)
print(sc)

### Predicting... ###
0.753731343284


In [87]:
#Correlação
c = datasetCopy.corr()
c

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.338481,-0.543351,-0.069809,-0.035322,0.081629,0.257307,-0.176509
Pclass,-0.338481,1.000000,0.131900,-0.331339,0.083081,0.018443,-0.549500,0.173511
Sex,-0.543351,0.131900,1.000000,0.084153,-0.114631,-0.245489,-0.182333,0.118492
Age,-0.069809,-0.331339,0.084153,1.000000,-0.232625,-0.179191,0.091566,-0.039610
SibSp,-0.035322,0.083081,-0.114631,-0.232625,1.000000,0.414838,0.159651,0.071480
Parch,0.081629,0.018443,-0.245489,-0.179191,0.414838,1.000000,0.216225,0.043351
Fare,0.257307,-0.549500,-0.182333,0.091566,0.159651,0.216225,1.000000,-0.230365
Embarked,-0.176509,0.173511,0.118492,-0.039610,0.071480,0.043351,-0.230365,1.000000


In [88]:
#Normalizando dados
dNorm = pd.DataFrame(preprocessing.scale(dataset))

In [89]:
dNorm.head(7)

,0,1,2,3,4,5,6
0,0.827377,0.737695,-0.592481,0.432793,-0.473674,-0.502445,0.587966
1,-1.566107,-1.355574,0.638789,0.432793,-0.473674,0.786845,-1.912644
2,0.827377,-1.355574,-0.284663,-0.474545,-0.473674,-0.488854,0.587966
3,-1.566107,-1.355574,0.407926,0.432793,-0.473674,0.420730,0.587966
4,0.827377,0.737695,0.407926,-0.474545,-0.473674,-0.486337,0.587966
5,0.827377,0.737695,0.000000,-0.474545,-0.473674,-0.478116,-0.662339
6,-1.566107,0.737695,1.870059,-0.474545,-0.473674,0.395814,0.587966


In [90]:
    dNorm[0] *= np.absolute(c['Survived']['Pclass'])
    dNorm[1] *= np.absolute(c['Survived']['Sex'])
    dNorm[2] *= np.absolute(c['Survived']['Age'])
    dNorm[3] *= np.absolute(c['Survived']['SibSp'])
    dNorm[4] *= np.absolute(c['Survived']['Parch'])
    dNorm[5] *= np.absolute(c['Survived']['Fare'])
    dNorm[6] *= np.absolute(c['Survived']['Embarked'])

In [91]:
dNorm.head(7)

,0,1,2,3,4,5,6
0,0.280052,0.400828,-0.041360,0.015287,-0.038666,-0.129282,0.103781
1,-0.530097,-0.736553,0.044593,0.015287,-0.038666,0.202460,-0.337599
2,0.280052,-0.736553,-0.019872,-0.016762,-0.038666,-0.125785,0.103781
3,-0.530097,-0.736553,0.028477,0.015287,-0.038666,0.108257,0.103781
4,0.280052,0.400828,0.028477,-0.016762,-0.038666,-0.125138,0.103781
5,0.280052,0.400828,0.000000,-0.016762,-0.038666,-0.123022,-0.116909
6,-0.530097,0.400828,0.130546,-0.016762,-0.038666,0.101845,0.103781


In [92]:
X = dNorm
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25)

In [93]:
knn = KNNClassifier()
knn.fit(X_train, Y_train, k=10, distance='euclidean')
res = knn.predict(X_test)
sc = metrics.accuracy_score(y_pred = res, y_true = Y_test)
print(sc)

### Predicting... ###
0.811659192825


In [97]:
knn = KNNClassifier()
acc = []

for x in range(0,10):
    k = 1+x*2
    knn.fit(X_train, Y_train, k=k, distance='euclidean')
    res = knn.predict(X_test)
    sc = metrics.accuracy_score(y_pred = res, y_true = Y_test)
    print('K=', k, ' Score=', sc)
    acc.append(sc)

### Predicting... ###
K= 1  Score= 0.784753363229
### Predicting... ###
K= 3  Score= 0.816143497758
### Predicting... ###
K= 5  Score= 0.820627802691
### Predicting... ###
K= 7  Score= 0.829596412556
### Predicting... ###
K= 9  Score= 0.793721973094
### Predicting... ###
K= 11  Score= 0.811659192825
### Predicting... ###
K= 13  Score= 0.829596412556
### Predicting... ###
K= 15  Score= 0.80269058296
### Predicting... ###
K= 17  Score= 0.807174887892
### Predicting... ###
K= 19  Score= 0.811659192825


In [99]:
fig,ex = plt.subplots()
ex.plot([1+x*2 for x in range(0,10)],acc,'g',linewidth=4)
ex.set_ylabel('Acurracy')
ex.set_xlabel('Vizinhos')
plt.grid()
#plt.show()

In [100]:
maxAcc = max(acc)
mK = acc.index(maxAcc)*4 + 1

knn.fit(X_train, Y_train, k=mK, distance='euclidean')
res = knn.predict(X_test)
sc = metrics.accuracy_score(y_pred = res, y_true = Y_test)
sc

### Predicting... ###


0.82959641255605376

In [44]:
print(metrics.classification_report(res, Y_test))

             precision    recall  f1-score   support

          0       0.88      0.80      0.84       158
          1       0.60      0.74      0.66        65

avg / total       0.80      0.78      0.79       223

